<a href="https://colab.research.google.com/github/karasu1982/colab_notebook/blob/main/202510_Multivariate_Zero_Inflated_Causal_Model_for_Regional_Mobility_Restriction_Effects_on_Consumer_Spending.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import io, textwrap, os, sys, math, urllib.parse, datetime as dt
import numpy as np
import pandas as pd
import arviz as az
import pymc as pm

In [ ]:
# 乱数固定
RNG_SEED = 12
np.random.seed(RNG_SEED)

# === データURL（Opportunity Insights / GitHub Raw）===============================
# 出典: OI Economic Tracker（消費=Affinity, モビリティ=Google, 政策=Policy Milestones）
# 参照: https://opportunityinsights.org/data/ -> 各CSVリンク
BASE = "https://raw.githubusercontent.com/OpportunityInsights/EconomicTracker/main/data"

URL_SPEND_STATE = f"{BASE}/Affinity%20-%20State%20-%20Daily.csv"
URL_MOB_STATE   = f"{BASE}/Google%20Mobility%20-%20State%20-%20Daily.csv"
URL_POLICY      = f"{BASE}/Policy%20Milestones%20-%20State.csv"

# === データ取得 ===================================================================
spend = pd.read_csv(URL_SPEND_STATE)
mob   = pd.read_csv(URL_MOB_STATE)
pol   = pd.read_csv(URL_POLICY)

# 確認用（列名が更新される可能性があるため、先頭だけ表示）
print(spend.columns.tolist()[:20])
print(mob.columns.tolist()[:20])
print(pol.columns.tolist()[:20])

['year', 'month', 'day', 'statefips', 'freq', 'spend_all', 'spend_aap', 'spend_acf', 'spend_aer', 'spend_apg', 'spend_durables', 'spend_nondurables', 'spend_grf', 'spend_gen', 'spend_hic', 'spend_hcs', 'spend_inperson', 'spend_inpersonmisc', 'spend_remoteservices', 'spend_sgh']
['year', 'month', 'day', 'statefips', 'gps_retail_and_recreation', 'gps_grocery_and_pharmacy', 'gps_parks', 'gps_transit_stations', 'gps_workplaces', 'gps_residential', 'gps_away_from_home']
['statefips', 'statename', 'date', 'policy_description', 'schools_first_closed', 'nonessential_biz_first_closed', 'stayathome_first_start']


In [ ]:
spend.head()

,year,month,day,statefips,freq,spend_all,spend_aap,spend_acf,spend_aer,spend_apg,...,spend_sgh,spend_tws,spend_retail_w_grocery,spend_retail_no_grocery,spend_all_incmiddle,spend_all_q1,spend_all_q2,spend_all_q3,spend_all_q4,provisional
0,2018,12,31,1,d,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
1,2018,12,31,2,d,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
2,2018,12,31,4,d,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
3,2018,12,31,5,d,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0
4,2018,12,31,6,d,.,.,.,.,.,...,.,.,.,.,.,.,.,.,.,0


In [ ]:
# === 前処理: 日付/州キーの統一 ===================================================
def unify_date(df, y='year', m='month', d='day'):
    # OIは year, month, day の3列持ちがち
    if all(c in df.columns for c in [y,m,d]):
        df['date'] = pd.to_datetime(dict(year=df[y], month=df[m], day=df[d]))
    elif 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
    else:
        raise ValueError("date列が見つからない/組み立て不可")
    return df

for _df in (spend, mob):
    cols = _df.columns
    y = 'year' if 'year' in cols else ('Year' if 'Year' in cols else None)
    m = 'month' if 'month' in cols else ('Month' if 'Month' in cols else None)
    d = 'day' if 'day' in cols else ('Day' if 'Day' in cols else None)
    unify_date(_df, y=y, m=m, d=d)

# 州キー（state / state_name / state_abbrなど）を推定
def find_state_col(df):
    for c in ['state', 'state_name', 'stateabbr', 'state_abbr', 'statefips', 'State', 'state_abbreviation']:
        if c in df.columns: return c
    raise ValueError("州を表す列が見つかりません")

STATE_COL_SPEND = find_state_col(spend)
STATE_COL_MOB   = find_state_col(mob)


In [ ]:
# === 変数選択: 消費カテゴリ =======================================================
# Affinity - State - Daily.csv には合計や業種別のインデックスが含まれます。
# 列名は更新され得るため、既知パターンからサブセットを推定します。
candidate_cols = [
    # よくある命名（例）
    'spend_all', 'spend_total', 'spend_all_scaled', 'spend_all_7dav',
    'spend_retail', 'spend_grocery', 'spend_restaurants_and_accommodations',
    'spend_entertainment', 'spend_transport', 'spend_healthcare'
]
spend_cols = [c for c in candidate_cols if c in spend.columns]
if not spend_cols:
    # フォールバック: "spend_"で始まる列から代表的に抽出
    spend_cols = [c for c in spend.columns if c.startswith('spend_')]
    # 代表3カテゴリ（多変量化）
    spend_cols = spend_cols[:3]

print("使用する消費列:", spend_cols)

使用する消費列: ['spend_all']


In [ ]:
# === 政策（移動制限）データ: Stay-at-Home等からバイナリ時系列を構築 ============
# Policy Milestones - State.csv の実データ列に対応
# ['statefips', 'statename', 'date', 'policy_description',
#  'schools_first_closed', 'nonessential_biz_first_closed', 'stayathome_first_start']

def parse_policy_binary(pol_df, policy_keywords=('stay', 'home', 'shelter', 'lockdown')):
    # ポリシー名列（説明文）を指定
    polname_col = 'policy_description' if 'policy_description' in pol_df.columns else None

    # 州列（正式名称）
    stcol = 'statename' if 'statename' in pol_df.columns else None

    # 開始日の列を明示的に指定
    if 'stayathome_first_start' in pol_df.columns:
        start_col = 'stayathome_first_start'
    else:
        # フォールバック（例外的に）
        start_candidates = [c for c in pol_df.columns if 'start' in c.lower()]
        start_col = start_candidates[0] if start_candidates else None

    # 終了日列は存在しないため None
    end_col = None

    # 文字列でstay-at-home系をフィルタ（policy_description内を検索）
    if polname_col and pol_df[polname_col].notna().any():
        mask = pol_df[polname_col].astype(str).str.lower().str.contains('|'.join(policy_keywords))
        pol_sh = pol_df[mask].copy()
    else:
        # policy_descriptionがNaNばかりなら全体を使用
        pol_sh = pol_df.copy()

    # 日付型に変換
    if start_col:
        pol_sh['start'] = pd.to_datetime(pol_sh[start_col], errors='coerce')
    else:
        raise ValueError("開始日列（stayathome_first_start）が見つかりません。")

    # 終了日は不明のため NaT に設定
    pol_sh['end'] = pd.NaT

    # 州ごとに最初の開始日を取得（重複州があれば最小日）
    agg = (pol_sh
           .groupby(stcol, dropna=False)
           .agg(start=('start', 'min'), end=('end', 'max'))
           .reset_index()
           .rename(columns={stcol: 'state'}))

    return agg

# 実行
pol_sh = parse_policy_binary(pol)

In [ ]:
def parse_policy_binary(pol_df):
    """
    pol_df 期待列: ['statefips','statename','date','policy_description',
                   'schools_first_closed','nonessential_biz_first_closed',
                   'stayathome_first_start']
    目的: 州ごとの stay-at-home の start / end を1行にまとめる
    """
    df = pol_df.copy()
    # 基本整形
    df.columns = df.columns.str.strip()
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    if 'statename' not in df.columns:
        raise ValueError("statename 列がありません。")

    # 関連行の抽出（開始/解除キーワード）
    desc_col = 'policy_description' if 'policy_description' in df.columns else None
    if desc_col is None:
        df_rel = df.copy()
    else:
        desc = df[desc_col].astype(str).str.lower()
        start_kw = r"(stay[-\s]?at[-\s]?home|shelter[-\s]?in[-\s]?place|lockdown)"
        mask_related = desc.str.contains(start_kw, na=False)
        df_rel = df[mask_related].copy()

    # 解除キーワードでフラグ
    end_kw = r"(lift|end|expire|reopen|rescinded|terminated|phase\s?out)"
    is_end = df_rel[desc_col].astype(str).str.lower().str.contains(end_kw, na=False) if desc_col else pd.Series(False, index=df_rel.index)

    # —— まずは同一内容の重複行を落とす（州・日付・開始/解除の別まで）——
    tmp = df_rel.assign(is_end=is_end)
    tmp = tmp.drop_duplicates(subset=['statename', 'date', 'is_end'])

    # 州ごとに start=最初に発令された日（解除でない行の最小日）
    starts = (tmp.loc[~tmp['is_end']]
                  .groupby('statename', dropna=False)['date']
                  .min()
                  .rename('start'))

    # 州ごとに end=最初に解除が言及された日（解除行の最小日）
    ends = (tmp.loc[tmp['is_end']]
                .groupby('statename', dropna=False)['date']
                .min()
                .rename('end'))

    # 州ごと1行にまとめる（start, end）
    out = (pd.concat([starts, ends], axis=1)
             .reset_index()
             .rename(columns={'statename': 'state'}))

    # フォールバック: start が欠損の州には stayathome_first_start を利用
    if 'stayathome_first_start' in df.columns:
        fb = (df[['statename', 'stayathome_first_start']]
              .rename(columns={'statename': 'state'}))
        # 文字/数値どちらにも対応
        fb['fallback_start'] = pd.to_datetime(
            fb['stayathome_first_start'].replace(["", " ", "0", 0, "0000-00-00"], pd.NA),
            errors='coerce'
        )
        fb = (fb.dropna(subset=['fallback_start'])
                .groupby('state', dropna=False)['fallback_start']
                .min()
                .reset_index())

        out = out.merge(fb, on='state', how='left')
        out['start'] = out['start'].fillna(out['fallback_start'])
        out = out.drop(columns=['fallback_start'])

    # 不整合の修正：end < start は end を NaT に
    out.loc[out['end'].notna() & out['start'].notna() & (out['end'] < out['start']), 'end'] = pd.NaT

    # —— 最終一意化（同じ state, start, end が重複しても1行に）——
    out = (out.sort_values(['state', 'start', 'end'])
              .drop_duplicates(subset=['state', 'start', 'end'])
              .reset_index(drop=True))

    return out

pol_sh = parse_policy_binary(pol)

/tmp/ipython-input-1646456246.py:22: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_related = desc.str.contains(start_kw, na=False)
/tmp/ipython-input-1646456246.py:27: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  is_end = df_rel[desc_col].astype(str).str.lower().str.contains(end_kw, na=False) if desc_col else pd.Series(False, index=df_rel.index)


In [ ]:
pol_sh.head()

,state,start,end
0,Alabama,2020-04-03,2020-04-30
1,Alaska,2020-03-28,2020-04-24
2,Arizona,2020-03-31,2020-05-15
3,California,2020-03-19,2021-01-25
4,Colorado,2020-03-24,2020-04-26


In [ ]:
# === パネル結合（州×日） =========================================================
# 期間は両データの交差に合わせる
min_date = max(spend['date'].min(), mob['date'].min())
max_date = min(spend['date'].max(), mob['date'].max())

# 分析対象の州・期間のグリッド
states = sorted(list(set(spend[STATE_COL_SPEND].unique()).intersection(set(mob[STATE_COL_MOB].unique()))))
panel = pd.MultiIndex.from_product([states, pd.date_range(min_date, max_date, freq='D')], names=['state','date'])
df = pd.DataFrame(index=panel).reset_index()

# 消費（複数カテゴリ）マージ
use_cols = [STATE_COL_SPEND, 'date'] + spend_cols
df = df.merge(spend[use_cols].rename(columns={STATE_COL_SPEND:'state'}), on=['state','date'], how='left')

# モビリティの代表（例：Retail & Recreation (% change from baseline)）
# 列名の候補から決める
mob_candidates = [c for c in mob.columns if any(k in c.lower() for k in ['retail', 'recreation', 'retail_and_recreation'])]
mob_col = mob_candidates[0] if mob_candidates else None
if mob_col:
    df = df.merge(mob[[STATE_COL_MOB,'date',mob_col]].rename(columns={STATE_COL_MOB:'state'}), on=['state','date'], how='left')
    df.rename(columns={mob_col:'mobility_rr'}, inplace=True)

# 政策フラグ（stay-at-home期間中=1）
def build_policy_flag(df, polwin):
    # Convert 'state' column in polwin to numeric to match df
    polwin['state'] = pd.to_numeric(polwin['state'], errors='coerce')
    df = df.merge(polwin, on='state', how='left')
    # Check if 'end' column exists before using it
    if 'end' in df.columns:
      df['policy_flag'] = ((df['date']>=df['start']) & (df['date']<=df['end'])).astype(float).fillna(0.0)
      df.drop(columns=['start','end'], inplace=True)
    else:
      # If 'end' column is missing, assume policy is in effect from 'start' date onwards
      df['policy_flag'] = (df['date']>=df['start']).astype(float).fillna(0.0)
      df.drop(columns=['start'], inplace=True)
    return df

df = build_policy_flag(df, pol_sh)

In [ ]:
df.head()

,state,date,spend_all,mobility_rr,policy_flag
0,1,2020-02-24,-.0302,0.00286,0.0
1,1,2020-02-25,-.039,0.01860,0.0
2,1,2020-02-26,-.0454,0.03570,0.0
3,1,2020-02-27,-.0393,0.06290,0.0
4,1,2020-02-28,-.0443,0.07570,0.0


In [ ]:
# 週単位に集約（週日曜終わり＝W-SUNなど。DIDに近い週頻度で安定化）
# Convert spend_cols and mobility_rr to numeric, coercing errors
for col in spend_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
if 'mobility_rr' in df.columns:
    df['mobility_rr'] = pd.to_numeric(df['mobility_rr'], errors='coerce')

agg_dict = {c:'mean' for c in spend_cols}
if 'mobility_rr' in df.columns: agg_dict['mobility_rr'] = 'mean'
agg_dict['policy_flag'] = 'max'  # その週に一度でも発令があれば1

dfw = (df
       .set_index('date')
       .groupby('state')
       .resample('W-SUN')
       .agg(agg_dict)
       .reset_index())

In [ ]:
# ゼロ判定（しきい値以下をゼロと見なす）
ZERO_EPS = 1e-6
for c in spend_cols:
    dfw[f'{c}_is_zero'] = (dfw[c].fillna(0.0).abs() <= ZERO_EPS).astype(int)
    # 正値パート用にクリッピング（負やNaNは落とす/小さい値は微小正に）
    dfw[f'{c}_pos'] = dfw[c].clip(lower=ZERO_EPS)

# 標準化用の補助（説明変数）
if 'mobility_rr' in dfw.columns:
    dfw['mobility_rr_std'] = (dfw['mobility_rr'] - dfw['mobility_rr'].mean())/dfw['mobility_rr'].std()

# 分析用にNA除去（必要最低限）
model_df = dfw.dropna(subset=['policy_flag'] + [f'{c}_pos' for c in spend_cols])

In [ ]:
# === インデクシング（州×週） =====================================================
states_list = sorted(model_df['state'].unique().tolist())
state_index = {s:i for i,s in enumerate(states_list)}
model_df['state_idx'] = model_df['state'].map(state_index).astype(int)

# カテゴリのインデックス化
K = len(spend_cols)

/tmp/ipython-input-1164909279.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['state_idx'] = model_df['state'].map(state_index).astype(int)


In [ ]:
model_df.head()

,state,date,spend_all,mobility_rr,policy_flag,spend_all_is_zero,spend_all_pos,mobility_rr_std,state_idx
0,1,2020-03-01,-0.039871,0.056394,0.0,0,0.000001,1.064682,0
1,1,2020-03-08,-0.011044,0.121571,0.0,0,0.000001,1.537313,0
2,1,2020-03-15,0.004854,0.092471,0.0,0,0.004854,1.326294,0
3,1,2020-03-22,-0.010457,-0.074614,0.0,0,0.000001,0.114676,0
4,1,2020-03-29,-0.243857,-0.307429,0.0,0,0.000001,-1.573572,0


In [ ]:
!pip install numpyro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 370.9/370.9 kB 8.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import arviz as az

# 事前準備
coords = {
    "obs": np.arange(len(model_df)),
    "state": states_list,
    "cat": spend_cols,
}
K = len(spend_cols)

with pm.Model(coords=coords) as model:
    # ===== データ =====
    state_idx = pm.Data("state_idx", model_df['state_idx'].values, dims="obs")
    policy    = pm.Data("policy",    model_df['policy_flag'].values, dims="obs")
    if 'mobility_rr_std' in model_df.columns:
        mobx = pm.Data("mobx", model_df['mobility_rr_std'].values, dims="obs")

    # 観測Y（pos: 正値、zero: 0/1フラグ）
    Y_pos  = np.column_stack([model_df[f'{c}_pos'].values     for c in spend_cols])
    Y_zero = np.column_stack([model_df[f'{c}_is_zero'].values for c in spend_cols])

    # LogNormal観測のため >0 を保証（ゼロ部で分けていても保険でクリップ）
    Y_pos = np.clip(Y_pos, 1e-12, None)

    # ===== Part A: Zero（Bernoulli, ロジスティック）=====
    alpha_zero = pm.Normal("alpha_zero", 0.0, 1.0)
    beta_zero  = pm.Normal("beta_zero",  0.0, 1.0)
    if 'mobility_rr_std' in model_df.columns:
        gamma_zero = pm.Normal("gamma_zero", 0.0, 1.0)

    sigma_state_zero = pm.HalfNormal("sigma_state_zero", 1.0)
    z_state = pm.Normal("z_state", 0.0, 1.0, dims="state")
    re_state_zero = pm.Deterministic("re_state_zero", z_state * sigma_state_zero, dims="state")

    delta_zero = pm.Normal("delta_zero", 0.0, 1.0, dims="cat")

    # broadcastable な軸追加（重要）
    policy_bc        = pt.shape_padaxis(policy, 1)                      # (n,1)
    re_state_zero_bc = pt.shape_padaxis(re_state_zero[state_idx], 1)    # (n,1)
    delta_zero_bc    = pt.shape_padaxis(delta_zero, 0)                  # (1,K)
    if 'mobility_rr_std' in model_df.columns:
        mobx_bc = pt.shape_padaxis(mobx, 1)                             # (n,1)

    lin_zero = alpha_zero + beta_zero * policy_bc
    if 'mobility_rr_std' in model_df.columns:
        lin_zero = lin_zero + gamma_zero * mobx_bc
    lin_zero = lin_zero + re_state_zero_bc + delta_zero_bc              # (n,K)

    p_zero = pm.Deterministic("p_zero", pm.math.sigmoid(lin_zero), dims=("obs","cat"))
    pm.Bernoulli("Y_zero", p=p_zero, observed=Y_zero, dims=("obs","cat"))

    # ===== Part B: Positive（LogNormal, 多変量ランダム効果）=====
    alpha_pos = pm.Normal("alpha_pos", 0.0, 2.0, dims="cat")            # (K,)
    beta_pos  = pm.Normal("beta_pos",  0.0, 1.0, dims="cat")            # (K,)
    if 'mobility_rr_std' in model_df.columns:
        gamma_pos = pm.Normal("gamma_pos", 0.0, 1.0, dims="cat")        # (K,)

    # 州×カテゴリの相関付きランダム効果
    chol, corr, sigmas = pm.LKJCholeskyCov(
        "chol_cov_state", n=K, eta=2.0, sd_dist=pm.HalfNormal.dist(1.0), compute_corr=True
    )
    z_state_cat = pm.Normal("z_state_cat", 0.0, 1.0, dims=("state","cat"))  # (S,K)
    re_state_cat = pm.Deterministic("re_state_cat", z_state_cat @ chol.T, dims=("state","cat"))  # (S,K)

    # broadcastable 揃え
    alpha_pos_bc = pt.shape_padaxis(alpha_pos, 0)      # (1,K)
    beta_pos_bc  = pt.shape_padaxis(beta_pos,  0)      # (1,K)
    if 'mobility_rr_std' in model_df.columns:
        gamma_pos_bc = pt.shape_padaxis(gamma_pos, 0)  # (1,K)

    policy_bc = pt.shape_padaxis(policy, 1)            # (n,1)
    mu_pos = alpha_pos_bc + beta_pos_bc * policy_bc    # (n,K)
    if 'mobility_rr_std' in model_df.columns:
        mobx_bc = pt.shape_padaxis(mobx, 1)            # (n,1)
        mu_pos = mu_pos + gamma_pos_bc * mobx_bc       # (n,K)
    mu_pos = mu_pos + re_state_cat[state_idx, :]       # (n,K)  ログ空間の平均

    sigma_obs = pm.HalfNormal("sigma_obs", 1.0, dims="cat")  # ログ空間の標準偏差
    sigma_obs_bc = pt.shape_padaxis(sigma_obs, 0)            # (1,K)

    # ここがA方式の肝：観測は生の正値を渡す
    pm.LogNormal(
        "Y_pos",
        mu=mu_pos,
        sigma=sigma_obs_bc,
        observed=Y_pos,         # ← np.log は渡さない
        dims=("obs","cat"),
    )

    # ACE（正値部・カテゴリ別）
    ace_pos = pm.Deterministic("ACE_pos", beta_pos, dims="cat")

    idata = pm.sample(
        1000, tune=1000, chains=2, target_accept=0.9,
        random_seed=RNG_SEED, nuts_sampler="numpyro", progressbar=True,
        init="jitter+adapt_diag"   # 初期安定化
    )

az.summary(idata, var_names=["alpha_pos","beta_pos","sigma_obs","ACE_pos"])

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha_pos[spend_all],-5.455,0.270,-5.971,-4.990,0.017,0.009,244.0,398.0,1.0
beta_pos[spend_all],0.017,1.042,-1.923,2.015,0.016,0.034,4082.0,1333.0,1.0
sigma_obs[spend_all],4.423,0.040,4.356,4.501,0.001,0.001,5704.0,1254.0,1.0
ACE_pos[spend_all],0.017,1.042,-1.923,2.015,0.016,0.034,4082.0,1333.0,1.0
